In [1]:
import numpy as np
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.mllib.recommendation import ALS
from pyspark.mllib.recommendation import Rating

In [2]:
conf=SparkConf().setAppName("RatingsApp")
sc=SparkContext (conf=conf)
ratings_data = sc.textFile('C://spark/ratings.csv') #movie lens small latest dataset

In [3]:
secondParameter = 5029
(trainingRatings, testRatings) = ratings_data.randomSplit([0.7, 0.3], secondParameter) 

In [4]:
trainingRatings.first()

'1,1,4.0,964982703'

In [5]:
testRatings.first()

'1,3,4.0,964981247'

In [6]:
trainingData = trainingRatings.map(lambda l: l.split(',')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [7]:
trainingData.first()

Rating(user=1, product=1, rating=4.0)

In [8]:
testData = testRatings.map(lambda l: l.split(',')).map(lambda l: (int(l[0]), int(l[1])))

In [9]:
testData.first()

(1, 3)

In [10]:
model = ALS.train(trainingData, 10, 10, 0.01)

In [11]:
model.predict(56, 46)

4.9012410562665965

In [12]:
predictions = model.predictAll(testData)

In [13]:
predictions.first()

Rating(user=4, product=1084, rating=5.107646856215513)

In [14]:
predictions=predictions.map(lambda r: ((r[0],r[1]),r[2]))
predictions.take(3)

[((4, 1084), 5.107646856215513),
 ((600, 1084), 3.7262849542824314),
 ((140, 1084), 3.187012888185255)]

In [15]:
testRatings = testRatings.map(lambda l: l.split(',')).map(lambda l: ((int(l[0]), int(l[1])), float(l[2])))
testRatings.take(5)

[((1, 3), 4.0),
 ((1, 6), 4.0),
 ((1, 101), 5.0),
 ((1, 110), 4.0),
 ((1, 157), 5.0)]

In [16]:
ratingsAndPredictions = testRatings.join(predictions)
ratingsAndPredictions.take(5)

[((1, 3), (4.0, 2.9659739752605443)),
 ((1, 333), (5.0, 4.023078378280333)),
 ((1, 527), (5.0, 4.63878497521177)),
 ((1, 919), (5.0, 4.320842758780589)),
 ((1, 1023), (5.0, 3.1368863118779124))]

In [17]:
MSE = ratingsAndPredictions.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print ("Mean Squared Error = " + str(MSE))

Mean Squared Error = 1.3432699861220738
